In [72]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [73]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000016DCFDCEB90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016DCFDCDA50>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [74]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hey My Name is Abhishek and I am a student at BITS Pilani,Hyderabad Campus.")])

AIMessage(content="Hi Abhishek!\n\nIt's great to meet you. BITS Pilani, Hyderabad Campus is a fantastic institution.  \n\nWhat are you studying there? Are you enjoying your time at BITS?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 27, 'total_tokens': 69, 'completion_time': 0.076363636, 'prompt_time': 0.00141288, 'queue_time': 0.248440908, 'total_time': 0.077776516}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--49a85f8d-8770-49af-8278-31c33b1f277a-0', usage_metadata={'input_tokens': 27, 'output_tokens': 42, 'total_tokens': 69})

In [75]:
## To make sure the model can respond to follow-up questions
from langchain_core.messages import AIMessage
model.invoke( 
    [ 
        HumanMessage(content="Hey My Name is Abhishek and I am a student at BITS Pilani,Hyderabad Campus."),
        AIMessage(content="Hey Abhishek,\n\nNice to meet you! It's great to know you're a student at BITS Pilani, Hyderabad Campus. \n\nWhat are you studying there? Are you enjoying your time at BITS Pilani?"),
        HumanMessage(content="Hey Whats my name and where do I study?")
    ]
)

AIMessage(content="You said your name is Abhishek and you study at BITS Pilani, Hyderabad Campus!  \n\nIs there anything else you'd like to talk about? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 93, 'total_tokens': 129, 'completion_time': 0.065454545, 'prompt_time': 0.002576969, 'queue_time': 0.249445402, 'total_time': 0.068031514}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--993f0ac4-7a17-40d9-abb8-6e54d1365646-0', usage_metadata={'input_tokens': 93, 'output_tokens': 36, 'total_tokens': 129})

In [76]:
### Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [77]:
config = {"configurable":{"session_id": "chat1"}}

In [78]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My Name is Abhishek and I am a student at BITS Pilani,Hyderabad Campus.")],
    config=config
)

In [79]:
response.content

"Hey Abhishek!\n\nThat's cool! BITS Pilani, Hyderabad Campus is a great place. \n\nWhat are you studying there? \n\nAre you enjoying your time at BITS Pilani?\n"

In [80]:
with_message_history.invoke(
    [HumanMessage(content="Hey Whats my name and where do I study?")],
    config=config
)

AIMessage(content="You said your name is Abhishek and you study at BITS Pilani, Hyderabad Campus!  \n\nIs there anything else you'd like to tell me? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 88, 'total_tokens': 124, 'completion_time': 0.065454545, 'prompt_time': 0.00251793, 'queue_time': 0.24719418, 'total_time': 0.067972475}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b63d4826-c503-4bc7-8cfa-f80b0a6925da-0', usage_metadata={'input_tokens': 88, 'output_tokens': 36, 'total_tokens': 124})

In [81]:
##Change the config to a different session_id
config1= {"configurable":{"session_id": "chat2"}}
 
response=with_message_history.invoke(
    [HumanMessage(content="Hey Whats my name and where do I study?")],
    config=config1
)

response.content

"As an AI, I have no memory of past conversations and no access to personal information about you, including your name or where you study. If you'd like to tell me, I'm happy to know! 😊\n"

In [82]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My Name is Paul")],
    config=config1
)

response.content

'Hi Paul! Nice to meet you. 😊 \n\nIs there anything I can help you with today?\n'

In [83]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey Whats my name")],
    config=config1
)

response.content

'Your name is Paul!  😊  I remember. \n\nAnything else I can help you with?\n'

## Prompt Templates

In [84]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant, Answer all the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
        
    ]
)

chain=prompt | model

In [85]:
chain.invoke({"messages":[HumanMessage(content="Hi My Name is Abhishek")]})

AIMessage(content="Hi Abhishek, it's nice to meet you! \n\nHow can I help you today? 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 31, 'total_tokens': 57, 'completion_time': 0.047272727, 'prompt_time': 0.00149344, 'queue_time': 0.24726161, 'total_time': 0.048766167}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--44cd5bba-a8d3-45aa-9b85-acdd2da5dc6a-0', usage_metadata={'input_tokens': 31, 'output_tokens': 26, 'total_tokens': 57})

In [86]:
with_message_history= RunnableWithMessageHistory(chain,get_session_history)

In [87]:
config= {"configurable":{"session_id": "chat3"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi My Name is Abhishek")]},
    config=config
)

response.content

"Hello Abhishek! It's nice to meet you. 😊\n\nI'm ready to answer your questions. What can I help you with today?  \n\n"

In [88]:
## ADD MORE COMPLEXITY TO THE PROMPT

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant, Answer all the questions to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages"),
        
    ]
)

chain=prompt | model

In [89]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My Name is Abhishek")], "language": "Telugu"})
response.content

"మీరు Abhishek అని పేరు మీకు కలం గా ఉంది. నేను మీకు సహాయాన్ని అందించడానికి సిద్ధంగా ఉన్నాను. మీరు నాకు ఏదైనా అడిగితే, నా శక్తి అవధులలో నేను మీకు సమాధానం ఇస్తాను. 😊\n\n(Translation: Your name is Abhishek, I'm happy to know that. I'm ready to help you. Ask me anything, and I will answer to the best of my ability.) \n"

In [90]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")


In [91]:
config= {"configurable":{"session_id": "chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi My Name is Abhishek")],"language": "Telugu"},
    config=config
) 

response.content

'నమస్తే Abhishek! 😊 \n\nనాకు మీ ప్రశ్నలకు ఉత్తరం చెప్పడంలో సంతోషం. మీరు ఏం అడిగితే, నేను నా శక్తి వరకు సమాధానం ఇస్తాను. \n\n'

### Managing the conversation history
'trim_messages'


In [92]:
from langchain_core.messages.utils import trim_messages
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

trimmer=trim_messages(
    max_tokens=200,
    strategy="last",
    token_counter=model.get_num_tokens_from_messages,
    include_system=True,
    allow_partial=True,
    start_on="human"
)

messages=[
    SystemMessage(content="You are a helpful assistant, Answer all the questions to the best of your ability"),
    HumanMessage(content="Hey My Name is Abhishek and I am a student at BITS Pilani,Hyderabad Campus."),
    AIMessage(content="Hey Abhishek,\n\nNice to meet you! It's great to know you're a student at BITS Pilani, Hyderabad Campus. \n\nWhat are you studying there? Are you enjoying your time at BITS Pilani?"),
    HumanMessage(content="I am studying Mechanical Engineering and I am enjoying my time at BITS Pilani."),
    AIMessage(content="That's great to hear! Mechanical Engineering is a fascinating field. \n\nWhat specific areas of Mechanical Engineering are you most interested in? Do you have any favorite projects or experiences at BITS Pilani so far?"),
    HumanMessage(content="I am interested in AI and I have worked on a few projects related to it. I also enjoy the campus life at BITS Pilani."),
    AIMessage(content="AI is an exciting area to explore, especially in the context of Mechanical Engineering. \n\nIt's great that you're working on projects related to it. BITS Pilani has a vibrant campus life, and I'm sure you're making the most of it! \n\nIf you have any specific questions or topics you'd like to discuss, feel free to ask."),
]

trimmer.invoke(messages)

[SystemMessage(content='You are a helpful assistant, Answer all the questions to the best of your ability', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I am studying Mechanical Engineering and I am enjoying my time at BITS Pilani.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's great to hear! Mechanical Engineering is a fascinating field. \n\nWhat specific areas of Mechanical Engineering are you most interested in? Do you have any favorite projects or experiences at BITS Pilani so far?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I am interested in AI and I have worked on a few projects related to it. I also enjoy the campus life at BITS Pilani.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="AI is an exciting area to explore, especially in the context of Mechanical Engineering. \n\nIt's great that you're working on projects related to it. BITS Pilani has a vibrant campus life, and I'm sure yo

In [93]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages") |trimmer)
    |prompt | model
)

response=chain.invoke({
    "messages": messages + [HumanMessage(content="What am I studying")],
    "language": "English"}
)

response.content

'You are studying **Mechanical Engineering** at BITS Pilani.  \n\nYou mentioned it at the beginning of our conversation! 😊 \n\n\n\n'

In [101]:
### Lets wrap this in the message history
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id": "chat5"}}

In [102]:
response=with_message_history.invoke({
    "messages": messages + [HumanMessage(content="What am I studying")],
    "language": "English"},
    config=config
)

response.content

"You said you are studying **Mechanical Engineering** at BITS Pilani.  \n\n\nLet me know if you have any other questions about your studies or anything else! I'm here to help.\n"

### Vector Stores and Retreivers


In [103]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "dog_facts.txt"}
    ),
    Document(
        page_content="Cats are independent animals, often known for their playful and curious nature.",
        metadata={"source": "cat_facts.txt"}                
    ),
    Document(
        page_content="Birds are fascinating creatures, often admired for their ability to fly and their diverse species.",
        metadata={"source": "bird_facts.txt"}       
    ),
    Document(
        page_content="Fish are aquatic animals, known for their gills and fins, and they come in various species.",
        metadata={"source": "fish_facts.txt"}
    ),
    Document(
        page_content="Reptiles are cold-blooded animals, often characterized by their scaly skin and ability to live in diverse environments.",
        metadata={"source": "reptile_facts.txt"}
    )
]


In [104]:
documents

[Document(metadata={'source': 'dog_facts.txt'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'cat_facts.txt'}, page_content='Cats are independent animals, often known for their playful and curious nature.'),
 Document(metadata={'source': 'bird_facts.txt'}, page_content='Birds are fascinating creatures, often admired for their ability to fly and their diverse species.'),
 Document(metadata={'source': 'fish_facts.txt'}, page_content='Fish are aquatic animals, known for their gills and fins, and they come in various species.'),
 Document(metadata={'source': 'reptile_facts.txt'}, page_content='Reptiles are cold-blooded animals, often characterized by their scaly skin and ability to live in diverse environments.')]

In [105]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm=ChatGroq(groq_api_key=groq_api_key,model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000016DCFD3CA30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016DD0C94100>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2")

NameError: name 'LRScheduler' is not defined

In [ ]:
##VectorStores
from langchain_chroma import Chroma
